In [ ]:
import pymongo
from pymongo import MongoClient
import pandas as pd
from dotenv import load_dotenv
import os
import pprint

In [2]:
load_dotenv(dotenv_path = "password.env") 

# MONGO_URI example: MONGO_URI=mongodb://localhost:27017/

mongo_uri = os.getenv("mongo_uri")
mongo_database = os.getenv("mongo_database")

client = MongoClient(mongo_uri)

db = client[mongo_database]

In [32]:
# Loading in csv data
patrons_data = pd.read_csv('patrons.csv')
inventory = pd.read_csv('inventory.csv')
books = pd.read_csv('books.csv')
authors = pd.read_csv('authors.csv')
loans = pd.read_csv('loans.csv')
books['isbn'] = books['isbn'].astype(str)
inventory['isbn'] = inventory['isbn'].astype(str)
authors['author_id'] = authors['author_id'].astype('int64')
books['author_id'] = books['author_id'].astype('int64')


# Converting to dictionaries
patrons_dict = patrons_data.to_dict('records')
loans_dict = loans.to_dict('records')
books_dict = books.to_dict('records')
inventory_dict = inventory.to_dict('records')
authors_dict = authors.to_dict('records')

# Adding authors as a subset of books
for item in books_dict:
    item['author'] = {}
for item in authors_dict:
    author_id = item['author_id']
    for book in books_dict:
        if book['author_id'] == author_id:
            book['author'] = item

# Merging books' meta data with inventoried books
for item in books_dict:
    isbn = item['isbn']
    for book in inventory_dict:
        if book['isbn'] == isbn:
            book['title'] = item['title']
            book['pub_year'] = item['pub_year']
            book['genre'] = item['genre']
            book['overdue_fee_rate'] = item['overdue_fee_rate']
            book['author'] = item['author']

# Adding inventories books as a subset of loans
for item in loans_dict:
    item['book'] = {}
for item in inventory_dict:
    book_id = item['book_id']
    for loan in loans_dict:
        if loan['book_id'] == book_id:
            loan['book'] = item
    item.pop('book_id')

# Inserting total overdue fees per patron
for patron in patrons_dict:
    patron['loan_history'] = []
    patron['total_fees_owed'] = 0
    for item in loans_dict:
        days = item['days_overdue']
        if pd.isna(days):
            days = 0
        rate = item['book']['overdue_fee_rate']
        fee = rate * days
        if item['patron_id'] == patron['patron_id']:
            patron['total_fees_owed'] += fee

# Adding loans as a subset of patrons
for item in loans_dict:
    patron_id = item['patron_id']
    for patron in patrons_dict:
        if patron['patron_id'] == patron_id:
            patron['loan_history'].append(item)
            item.pop("loan_id")
            item.pop("book_id")
            item.pop('patron_id')
            break


# Getting rid of `patron_id` variable in patrons
for item in patrons_dict:
    item.pop('patron_id', None)

patrons_dict[-4]


{'first_name': 'Chloe',
 'last_name': 'Robinson',
 'address': '305 Redwood Circle',
 'phone_number': '101-010-1010',
 'age': 29,
 'loan_history': [{'loan_date': '2024-12-27',
   'due_date': '2025-01-26',
   'return_date': '2025-01-29',
   'days_overdue': 3.0,
   'book': {'isbn': '9780374528379',
    'condition': 'Fair',
    'title': 'Interpreter of Maladies',
    'pub_year': 1999,
    'genre': 'Short Stories',
    'overdue_fee_rate': 0.25,
    'author': {'author_id': 19,
     'first_name': 'Jhumpa',
     'last_name': 'Lahiri',
     'nationality': 'Indian-American',
     'language': 'English',
     'birth_year': '1967',
     'biography': "Jhumpa Lahiri won the Pulitzer Prize for 'Interpreter of Maladies'."}}},
  {'loan_date': '2025-05-22',
   'due_date': '2025-06-21',
   'return_date': nan,
   'days_overdue': nan,
   'book': {'isbn': '9780812981602',
    'condition': 'Fair',
    'title': 'The Vegetarian',
    'pub_year': 2007,
    'genre': 'Literary Fiction',
    'overdue_fee_rate': 0.5

In [35]:
# PATRONS COLLECTION

# Adding patron objects to MongoDB
patrons = db.patrons
patrons.insert_many(patrons_dict)

# Delete current objects to repopulate if necessary
# patrons.delete_many({})

InsertManyResult([ObjectId('683865ba8246f9e67a45ab44'), ObjectId('683865ba8246f9e67a45ab45'), ObjectId('683865ba8246f9e67a45ab46'), ObjectId('683865ba8246f9e67a45ab47'), ObjectId('683865ba8246f9e67a45ab48'), ObjectId('683865ba8246f9e67a45ab49'), ObjectId('683865ba8246f9e67a45ab4a'), ObjectId('683865ba8246f9e67a45ab4b'), ObjectId('683865ba8246f9e67a45ab4c'), ObjectId('683865ba8246f9e67a45ab4d'), ObjectId('683865ba8246f9e67a45ab4e'), ObjectId('683865ba8246f9e67a45ab4f'), ObjectId('683865ba8246f9e67a45ab50'), ObjectId('683865ba8246f9e67a45ab51'), ObjectId('683865ba8246f9e67a45ab52'), ObjectId('683865ba8246f9e67a45ab53'), ObjectId('683865ba8246f9e67a45ab54'), ObjectId('683865ba8246f9e67a45ab55'), ObjectId('683865ba8246f9e67a45ab56'), ObjectId('683865ba8246f9e67a45ab57'), ObjectId('683865ba8246f9e67a45ab58'), ObjectId('683865ba8246f9e67a45ab59'), ObjectId('683865ba8246f9e67a45ab5a'), ObjectId('683865ba8246f9e67a45ab5b'), ObjectId('683865ba8246f9e67a45ab5c'), ObjectId('683865ba8246f9e67a45ab

In [36]:
# BOOKS COLLECTION

# Adding book objects to MongoDB
books = db.books
books.insert_many(inventory_dict)

# Delete current objects to repopulate if necessary
# books.delete_many({})

InsertManyResult([ObjectId('683865bd8246f9e67a45ab65'), ObjectId('683865bd8246f9e67a45ab66'), ObjectId('683865bd8246f9e67a45ab67'), ObjectId('683865bd8246f9e67a45ab68'), ObjectId('683865bd8246f9e67a45ab69'), ObjectId('683865bd8246f9e67a45ab6a'), ObjectId('683865bd8246f9e67a45ab6b'), ObjectId('683865bd8246f9e67a45ab6c'), ObjectId('683865bd8246f9e67a45ab6d'), ObjectId('683865bd8246f9e67a45ab6e'), ObjectId('683865bd8246f9e67a45ab6f'), ObjectId('683865bd8246f9e67a45ab70'), ObjectId('683865bd8246f9e67a45ab71'), ObjectId('683865bd8246f9e67a45ab72'), ObjectId('683865bd8246f9e67a45ab73'), ObjectId('683865bd8246f9e67a45ab74'), ObjectId('683865bd8246f9e67a45ab75'), ObjectId('683865bd8246f9e67a45ab76'), ObjectId('683865bd8246f9e67a45ab77'), ObjectId('683865bd8246f9e67a45ab78'), ObjectId('683865bd8246f9e67a45ab79'), ObjectId('683865bd8246f9e67a45ab7a'), ObjectId('683865bd8246f9e67a45ab7b'), ObjectId('683865bd8246f9e67a45ab7c'), ObjectId('683865bd8246f9e67a45ab7d'), ObjectId('683865bd8246f9e67a45ab

In [37]:
# Patrons with the highest library fees

query1 = {'total_fees_owed': {'$gte': 2}}
result = patrons.find(query1).sort('total_fees_owed', -1)

print("Patrons With the Highest Library Fees:")
for e in result:
    print(f"{e['first_name']} {e['last_name']}: ${e['total_fees_owed']}")


Patrons With the Highest Library Fees:
Olivia Smith: $12.0
Grace Clark: $6.75
James Davis: $5.0
Alexander Moore: $2.75
Mia Hernandez: $2.5
Sophia Miller: $2.25
Noah Brown: $2.0


In [94]:
# Every book with number of books

query2= books.aggregate(
    [{
    "$group" : 
        {"_id" : "$title", 
         "count" : {"$sum" : 1}
         }}
    ])

query2_list = list(query2)

for e in query2_list:
    e["title"] = e.pop("_id")
    e['count'] = e.pop('count')

df = pd.DataFrame(query2_list)
df = df.sort_values('count', ascending = False)

df


,title,count
19,FORTNITE (Official): Battle Royale Survival Guide,9
30,Half of a Yellow Sun,5
5,Americanah,4
1,The Book of Laughter and Forgetting,3
14,East of Eden,3
0,Interpreter of Maladies,3
18,The Stranger,2
8,White Teeth,2
29,Meditations,2
10,Parable of the Sower,2


In [92]:
# Most borrowed books

query3 = [
    {"$unwind": "$loan_history"},
    {"$group": {
        "_id": "$loan_history.book.title",
        "borrow_count": {"$sum": 1}
    }},
    {"$sort": {"borrow_count": -1}}
]

result = list(patrons.aggregate(query3))
for doc in result:
    doc['title'] = doc.pop('_id')
    doc['borrow_count'] = doc.pop('borrow_count')

df = pd.DataFrame(result)
df

,title,borrow_count
0,FORTNITE (Official): Battle Royale Survival Guide,7
1,The Brothers Karamazov,4
2,Half of a Yellow Sun,4
3,Interpreter of Maladies,3
4,The Book of Laughter and Forgetting,3
5,NW,3
6,Parable of the Sower,2
7,1984,2
8,The Stranger,2
9,Americanah,2


In [ ]:
# Most borrowed authors

query4 = [
    {"$unwind": "$loan_history"},
    {"$group": {
        "_id": "$loan_history.book.author.last_name",
        "borrow_count": {"$sum": 1}
    }},
    {"$sort": {"borrow_count": -1}}
]

result = list(patrons.aggregate(query4))
for doc in result:
    doc['author'] = doc.pop('_id')
    doc['borrow_count'] = doc.pop('borrow_count')

df = pd.DataFrame(result)
df

,author,borrow_count
0,Epic Games,7
1,Ngozi Adichie,6
2,Dostoevsky,6
3,Kundera,5
4,Smith,4
5,Orwell,4
6,Lahiri,3
7,Ishiguro,2
8,Atwood,2
9,Kang,2


In [ ]:
# Most popular genres

query5 = [
    {'$unwind': '$loan_history'},
    {'$group' : 
     {'_id' : '$loan_history.book.genre',
      'borrow_count' : {'$sum' : 1}}},
    {'$sort' : {'borrow_count' : -1}}
]

result = list(patrons.aggregate(query5))

for doc in result:
    doc['genre'] = doc.pop('_id')
    doc['borrow_count'] = doc.pop('borrow_count')

df = pd.DataFrame(result)
df

,genre,borrow_count
0,Philosophy,9
1,Self-help,8
2,Classic,7
3,Contemporary,6
4,Dystopian,4
5,Historical,4
6,Science Fiction,4
7,Literary Fiction,3
8,Short Stories,3
9,Historical Fiction,1
